## Imports

In [1]:
import sys, os
import numpy as np

# Import the rouskinhf package
sys.path.append('..')
from rouskinhf import import_dataset, DataFolder, setup_env

# make sure to source the env file
# OPTION 1: source the env file in the notebook
%load_ext dotenv
%dotenv env
# OPTION 2: source the env file in the terminal
setup_env(RNASTRUCTURE_PATH='/Users/yvesmartin/lib/RNAstructure/exe')

In [ ]:
p = '/Users/ymdt/data/shape_data.json'
import json
import numpy as np
data = json.load(open(p, 'r'))

data_copy = data.copy()
for ref, d in data.items():
    temp = d['dms']
    temp = [np.clip(x, 0, 1) if x >-1 else x for x in temp ]
    data_copy[ref]['dms'] = temp
    
json.dump(data_copy, open(p.replace('.json', '1.json'), 'w'))

# Create a datafolder from local files

These methods will allow you to process your data and create a datafolder from it. The accepted formats are:
- DREEM output
- fasta
- set of CTs
- already formatted json + info.json

**Make sure to change the paths to your own paths!**

### From DREEM output

In [ ]:
%reload_ext autoreload
%autoreload 2
datafolder = DataFolder.from_dreem_output(
    name='pri-miRNA', # name of the input data by default
    path_in='/Users/yvesmartin/Downloads/pri-miRNA.json', # path to the input data
    path_out='/Users/yvesmartin/src/rouskinhf/data/datafolders', 
    predict_structure=True,
    # predict_dms=False,
    tqdm=True,
    generate_npy=True
    )

### From a data.json file

In [20]:
%reload_ext autoreload
%autoreload 2
datafolder = DataFolder.from_data_json(
    name='ribonanza', # name of the input data by default
    path_in='/Users/yvesmartin/src/rouskinhf/data/ribonanza.json', 
    path_out='/Users/yvesmartin/src/rouskinhf/data/datafolders', 
    predict_structure=False,
    predict_dms=False,
    tqdm=True,
    generate_npy=True
    )

Parsing json file: 100%|██████████| 49001/49001 [00:03<00:00, 14230.55it/s]


Over a total of 49001 datapoints, there are:
    - 49001 valid datapoints
    - 0 invalid datapoints (ex: sequence with non-regular characters)
    - 0 datapoints with the same reference
    - 0 duplicate sequences with the same structure / dms
    - 0 duplicate sequences with different structure / dms


### From a list of CT files

In [ ]:
%reload_ext autoreload
%autoreload 2
datafolder = DataFolder.from_ct_folder(
    name='bpRNA', # name of the input data by default
    path_in='/Users/ymdt/Downloads/bpRNA', 
    path_out='/Users/ymdt/src/rouskinhf/data/datafolders', 
    predict_dms=False, # won't take the structure from the ct files into account
    tqdm=True, 
    generate_npy=True,
    )  


### From fasta

In [ ]:
datafolder = DataFolder.from_fasta(
    name= 'sequences', # name of the input data by default
    path_in = '/Users/ymdt/src/rouskinhf/data/input_files_for_testing/sequences.fasta', 
    path_out='/Users/ymdt/src/rouskinhf/data/datafolders', 
    predict_structure=True,
    predict_dms=True,
    tqdm=True,
    generate_npy=True,
    )
np.load(datafolder.get_dms_npy(), allow_pickle=True)

### From an existing local datafolder

In [ ]:
datafolder = DataFolder.from_local(
    name = '/Users/ymdt/src/rouskinhf/data/datafolders/for_testing',
)

# OR

datafolder = import_dataset(
    name = 'for_testing',
    data = 'DMS', # can be 'DMS' or 'structure'
    force_download=False # if True, will download the data even if it already exists locally
)

# Push a local datafolder to Hugging Face

### 1. Create a repository if it does not exist

In [21]:
# Find more arguments here: https://huggingface.co/docs/huggingface_hub/guides/repository#create-a-repository
datafolder.create_repo(
    exist_ok=True,
    private=True
)

### 2. Push the datafolder to Hugging Face

In [22]:

# Find more arguments here: https://huggingface.co/docs/huggingface_hub/guides/upload#upload-a-folder
future = datafolder.upload_folder(
    revision='main', # branch name
    commit_message='init commit',
    commit_description='',
    run_as_future=True,
)

future.done() # True if the upload is done
future.result() # Wait for the upload to complete (blocking action)

ribonanza.json:   0%|          | 0.00/251M [00:00<?, ?B/s]

data.json:   0%|          | 0.00/121M [00:00<?, ?B/s]

Upload 2 LFS files:   0%|          | 0/2 [00:00<?, ?it/s]

'https://huggingface.co/datasets/rouskinlab/ribonanza/tree/main/'

### 3. Check that the datafolder is on Hugging Face

Take a look at https://huggingface.co/rouskinlab

# Explore the datafolder object

In [ ]:
import numpy as np
import json
print(datafolder)
print('main folder:',datafolder.get_main_folder())
print('sequences.npy file:\n',np.load(datafolder.get_sequences_npy(), allow_pickle=True))
print('base_pairs.npy file:\n',np.load(datafolder.get_base_pairs_npy(), allow_pickle=True))
print('dms.npy file:\n',np.load(datafolder.get_dms_npy(), allow_pickle=True))
print('json file:\n', json.load(open(datafolder.get_json(), 'r')))
print('dms.npy file:',datafolder.get_dms_npy())
print('json file:',datafolder.get_json())
print('source files:',datafolder.get_source_files())
print('info file:',datafolder.get_info_file())

In [10]:
%reload_ext autoreload
%autoreload 2

data_pri = import_dataset(
    name = 'ribonanza',
    force_download=False
)

for k, v in data_pri.items():
    print(k, len(v))


# data_utr = import_dataset(
#     name = 'utr',
#     data = 'DMS', # can be 'DMS' or 'structure'
#     force_download=False # if True, will download the data even if it already exists locally
# )


Using local data for: ribonanza
references 49001
sequences 49001
base_pairs 49001
dms 49001
shape 49001


In [2]:
import torch
from rouskinhf.util import dot2int

def base_pairs_to_int_dot_bracket(base_pairs, sequence_length, dtype=torch.int64):
    dot_bracket = ["."] * sequence_length
    for i, j in base_pairs:
        dot_bracket[i] = "("
        dot_bracket[j] = ")"
    return torch.tensor([dot2int[s] for s in dot_bracket], dtype=dtype)

In [38]:
import lightning.pytorch as pl
from typing import Union
from torch.utils.data import DataLoader, random_split, Subset
import torch
import torch.nn.functional as F
from torch.utils.data import Dataset as TorchDataset
from torch import tensor, uint8, float32
import torch.nn.functional as F
from torch import nn
from rouskinhf import import_dataset
UKN = -1000.0

TEST_SETS_NAMES = {
    "structure": ["CT_files_pdbee"],
    'sequence': [],
    "dms": ["sarah_supermodel", "utr", "SARS2", "pri-miRNA"],
}

TEST_SETS_NAMES = [i for j in TEST_SETS_NAMES.values() for i in j]

class Dataset(TorchDataset):
    def __init__(self, name: str, force_download:bool, quality:float=1.) -> None:
        super().__init__()
        self.name = name
        data = import_dataset(name, force_download=force_download)

        # save data
        self.references = data["references"]
        self.sequences = data["sequences"]
        self.dms = data["dms"] if "dms" in data else None
        self.base_pairs = data["base_pairs"] if "base_pairs" in data else None
        self.shape = data["shape"] if "shape" in data else None
        self.quality = quality

    def __len__(self) -> int:
        return len(self.sequences)
    
    def __getitem__(self, index) -> tuple:
        
        sequence = tensor(self.sequences[index], dtype=uint8)
        
        def get_array(attr, dtype=float32):
            if not hasattr(self, attr):
                return None
            data = tensor(getattr(self, attr)[index], dtype=dtype)
            if torch.isnan(data).all():
                return None
            return data
        
        dms = get_array('dms')
        structure = get_array('base_pairs', dtype=uint8)
        shape = get_array('shape')
        
        data = {'sequence': sequence, 'dms': dms, 'structure': structure, 'shape': shape}
        drop_keys = [k for k, v in data.items() if v is None]
        for k in drop_keys:
            del data[k]
            
        metadata = {'reference': self.references[index], 'index': index, 'quality': self.quality}
        
        return data, metadata
    
    def collate_fn(self, batch):
        data, metadata = zip(*batch)
        
        # pad sequences 
        max_length = max([len(d['sequence']) for d in data])
        padding_values = {
            'sequence': 0,
            'dms': UKN,
            'structure': 'X',
            'shape': UKN,
        }
        
        for d in data:
            for k, v in d.items():
                d[k] = F.pad(v, (0, max_length - len(v)), value=padding_values[k])
                
        return data, metadata

class DataModule(pl.LightningDataModule):
    def __init__(
        self,
        name: Union[str, list],
        force_download=False,
        batch_size: int = 32,
        num_workers: int = 1,
        train_split: float = 0,
        valid_split: float = 0,
        predict_split: float = 0,
        overfit_mode=False,
        shuffle_train=True,
        shuffle_valid=False,
        **kwargs,
    ):
        """DataModule for the Rouskin lab datasets.

        Args:
            name: name of the dataset on the Rouskin lab HuggingFace repository. Can be a list of names.
            data: type of the data (e.g. 'dms', 'structure')
            force_download: re-download the dataset from the Rouskin lab HuggingFace repository
            batch_size: batch size for the dataloaders
            num_workers: number of workers for the dataloaders
            train_split: percentage of the dataset to use for training or number of samples to use for training. If None, the entire dataset minus the validation set is used for training
            valid_split: percentage of the dataset to use for validation or number of samples to use for validation
            predict_split: percentage of the dataset to use for prediction or number of samples to use for prediction
            zero_padding_to: pad sequences to this length. If None, sequences are not padded.
            overfit_mode: if True, the train set is used for validation and testing. Useful for debugging. Default is False.
        """
        # Save arguments
        super().__init__(**kwargs)

        if not self._use_multiple_datasets(name):
            self.name = [name]
        else: 
            self.name = name
            
        self.data = data
        self.force_download = force_download
        self.batch_size = batch_size
        self.num_workers = num_workers  
        self.splits = {
            "train": train_split,
            "valid": valid_split,
            "predict": predict_split,
        }
        self.shuffle = {
            "train": shuffle_train,
            "valid": shuffle_valid,
        }

        # we need to know the max sequence length for padding
        self.overfit_mode = overfit_mode
        self.setup()

        # Log hyperparameters
        train_split, valid_split, _ = self.size_sets
        self.save_hyperparameters(ignore=["force_download"])
        
    def _use_multiple_datasets(self, name):
        if type(name) == str:
            return False
        elif type(name) == list or type(name) == tuple:
            return True
        raise ValueError("name must be a string or a list of strings")

    def _dataset_merge(self, datasets):
        merge = datasets[0]
        collate_fn = merge.collate_fn
        for dataset in datasets[1:]:
            merge = merge + dataset
        merge.collate_fn = collate_fn
        return merge

    def setup(self, stage: str = None):
        
        dataFull = self._dataset_merge(
            [
                Dataset(
                    name=name,
                    force_download=self.force_download,
                )
                for name in self.name
            ]
        )
        self.collate_fn = dataFull.collate_fn
                    
        if stage == "fit" or stage is None:
            self.size_sets = _compute_size_sets(len(dataFull), train_split=self.splits['train'], valid_split=self.splits['valid'])  
            self.train_set, self.val_set, _ = random_split(dataFull, self.size_sets)

        if stage == "test" or stage is None:
            self.test_sets = self._select_test_dataset(
                force_download=self.force_download
            )

        if stage == "predict" or stage is None:
            self.predict_set = Subset(dataFull, range(0, round(len(dataFull) * self.splits['predict']) if type(self.splits['predict']) == float else self.splits['predict']))

            

    def _select_test_dataset(self, force_download=False):
        return [
            Dataset(
                name=name,
                force_download=force_download,
            )
            for name in TEST_SETS_NAMES
        ]
        

    def train_dataloader(self):
        return DataLoader(
            self.train_set,
            shuffle=self.shuffle["train"],
            collate_fn=self.collate_fn,
            **self.dataloader_args,
        )

    def val_dataloader(self):
        return DataLoader(
            self.val_set if not self.overfit_mode else self.train_set,
            shuffle=self.shuffle["valid"],
            collate_fn=self.collate_fn,
            **self.dataloader_args,
        )

    def test_dataloader(self):
        return [
            DataLoader(
                test_set,
                collate_fn=self.collate_fn,
                **self.dataloader_args,
            )
            for test_set in self.test_sets
        ]

    def predict_dataloader(self):
        return DataLoader(
            self.predict_set,
            collate_fn=self.collate_fn,
            **self.dataloader_args,
        )

    def teardown(self, stage: str):
        # Used to clean-up when the run is finished
        pass
            

def _compute_size_sets(len_data, train_split, valid_split):
    """Returns the size of the train and validation sets given the split percentages and the length of the dataset.

    Args:
        len_data: int
        train_split: float between 0 and 1, or integer, or None. If None, the train split is computed as 1 - valid_split. Default is None.
        valid_split: float between 0 and 1, or integer, or None. Default is 4000.

    Returns:
        train_set_size: int
        valid_set_size: int
        buffer_size: int

    Raises:
        AssertionError: if the split percentages do not sum to 1 or less, or if the train split is less than 0.

    Examples:
    >>> _compute_size_sets(100, 40, 0.2)
    (40, 20, 40)
    >>> _compute_size_sets(100, 40, 20)
    (40, 20, 40)
    >>> _compute_size_sets(100, None, 10)
    (90, 10, 0)
    >>> _compute_size_sets(100, 0.4, 0.2)
    (40, 20, 40)
    >>> _compute_size_sets(100, 0.4, 20)
    (40, 20, 40)
    """

    if valid_split <= 1 and type(valid_split) == float:
        valid_split = int(valid_split * len_data)

    if train_split is None:
        train_split = len_data - valid_split

    elif train_split <= 1 and type(train_split) == float:
        train_split = len_data - int((1 - train_split) * len_data)

    assert (
        train_split + valid_split <= len_data
    ), "The sum of the splits must be less than the length of the dataset"

    return train_split, valid_split, len_data - train_split - valid_split

datamodule = DataModule(
    name='ribonanza'
)

Using local data for: ribonanza
Using local data for: CT_files_pdbee
Using local data for: sarah_supermodel
Using local data for: utr
Using local data for: SARS2
Using local data for: pri-miRNA


In [31]:
%reload_ext autoreload
%autoreload 2


        
data = Dataset('ribonanza', force_download=False)
for d in data:
    print(d[0].keys())


Using local data for: ribonanza
dict_keys(['sequence', 'structure', 'shape'])
dict_keys(['sequence', 'dms', 'structure'])
dict_keys(['sequence', 'dms', 'structure'])
dict_keys(['sequence', 'dms', 'structure', 'shape'])
dict_keys(['sequence', 'dms', 'structure'])
dict_keys(['sequence', 'dms', 'structure', 'shape'])
dict_keys(['sequence', 'dms', 'structure'])
dict_keys(['sequence', 'dms', 'structure', 'shape'])
dict_keys(['sequence', 'structure', 'shape'])
dict_keys(['sequence', 'dms', 'structure'])
dict_keys(['sequence', 'dms', 'structure'])
dict_keys(['sequence', 'dms', 'structure'])
dict_keys(['sequence', 'dms', 'structure'])
dict_keys(['sequence', 'dms', 'structure', 'shape'])
dict_keys(['sequence', 'dms', 'structure'])
dict_keys(['sequence', 'dms', 'structure'])
dict_keys(['sequence', 'dms', 'structure'])
dict_keys(['sequence', 'dms', 'structure'])
dict_keys(['sequence', 'dms', 'structure'])
dict_keys(['sequence', 'dms', 'structure', 'shape'])
dict_keys(['sequence', 'dms', 'structur

In [23]:
torch.nan

nan

In [ ]:
import numpy as np
a = np.array([np.array([0,1,2,3]), np.array([2,3,4])], dtype=object)

In [ ]:
a = [1, 2]; b= [3, 4, 5]
np.array([np.array(a, dtype=np.float32), np.array(b, dtype=np.float32)], dtype=object)


In [ ]:

c, b = [0.1321452463246523462346532, 0.1321452463246523462346532], [0.1321452463246523462346532, 0.1321452463246523462346532]
np.array([np.array(c, dtype=np.float32), np.array(b, dtype=np.float32)], dtype=object)
